##Function
The objective of using this Jupyter notebook is experimentation and testing of the different models that will be developed in order to pick out the best one.


In [4]:
#Implementation 1
#Checking for performance after preprocessing 
from random import Random
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate



train_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/train_clean.csv")
test_df =  pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/test_clean.csv")

#X_train, y_train and X_test
y_train = train_df["bank_account"]
X_train = train_df.drop(["bank_account"], axis=1)
X_test = test_df.drop(["bank_account"], axis=1)

#Dict of classifiers
classifiers = {
    "KNN" : KNeighborsClassifier(),
    "LR" : LogisticRegression(max_iter=1000),
    "MLP" : MLPClassifier(max_iter=1000),
    "XGB" : XGBClassifier(),
    "LGB" : LGBMClassifier(),
    "SVC" : SVC(),
    "DT" : DecisionTreeClassifier(),
    "RT" : RandomForestClassifier()
}

results = pd.DataFrame(columns=["Classifier", "Avg_Accuracy", "Avg_F1_Score"])

for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(model, X_train, y_train, cv=10, 
                 scoring=(['accuracy', 'f1']))

    results = results.append({
        "Classifier": name, 
        "Avg_Accuracy": cv_results["test_accuracy"].mean(),
        "Avg_F1_Score" : cv_results["test_f1"].mean()
    }, ignore_index=True)

results["Avg_Overall"] = (results["Avg_Accuracy"] + results["Avg_F1_Score"])/2
results = results.sort_values("Avg_Overall", ascending=False)
results

/home/mwanikii/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/mwanikii/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

,Classifier,Avg_Accuracy,Avg_F1_Score,Avg_Overall
3,XGB,0.873536,0.451148,0.662342
2,MLP,0.871580,0.450744,0.661162
4,LGB,0.877149,0.441471,0.659310
1,LR,0.874172,0.410026,0.642099
7,RT,0.853176,0.423142,0.638159
6,DT,0.819296,0.391149,0.605222
0,KNN,0.858403,0.347177,0.602790
5,SVC,0.859208,0.000000,0.429604


In the above, XGB and LGB have displayed the best Avg_Overall score. In the next attempt, the models will be designed to take in categorical data as it is rather than encoding it.
NB: This is when the date column has not been removed

When the date column has been dropped, LGB is replaced by MLP while XGB maintains its position. 


In [23]:
#Implementation 2
#Performance without the removal of categorical features

<bound method NDFrame.head of        year  bank_account  location_type  cellphone_access  household_size  \
0      2018             1              0                 1               3   
1      2018             0              0                 0               5   
2      2018             1              1                 1               5   
3      2018             0              0                 1               5   
4      2018             0              1                 0               8   
...     ...           ...            ...               ...             ...   
23519  2018             0              0                 1               4   
23520  2018             0              0                 1               2   
23521  2018             0              0                 1               5   
23522  2018             0              1                 1               7   
23523  2018             0              0                 1              10   

       age_of_respondent  gender_